# Installation

In [7]:
# Mount gdrive for data sets
import os
from google.colab import drive
drive.mount('/gdrive')
try:
  os.symlink('/gdrive/My Drive/PHD/Tez', '/drive')
except FileExistsError:
  print("Seems like already mounted the gdrive, skipping the mounting part.")

# Download source code and copy to mounted drive
!rm -f master.zip
!wget https://github.com/aligokalppeker/hypelcnn/archive/refs/heads/master.zip
!unzip -d /drive/ -o -q master.zip && rm master.zip
!rm -rf /drive/hypelcnn/ && mv /drive/hypelcnn-master/ /drive/hypelcnn/

# Add source root to python PATH
import os
os.environ["PYTHONPATH"] += ":/drive/hypelcnn"
!echo $PYTHONPATH

# Python package setup
!apt-get --assume-yes install libcupti-dev
!pip install colour-science
!pip install tensorflow==1.15.5
!pip install tensorflow-gan==2.0.0
!pip install tensorflow-probability==0.8.0
!pip install gast==0.2.2
!pip install numpy==1.18.5

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Seems like already mounted the gdrive, skipping the mounting part.
--2022-08-08 08:47:21--  https://github.com/aligokalppeker/hypelcnn/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/aligokalppeker/hypelcnn/zip/refs/heads/master [following]
--2022-08-08 08:47:21--  https://codeload.github.com/aligokalppeker/hypelcnn/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ] 135.32K  --.-KB/s    in 0.02s   



# CNN Training and Visualization

In [ ]:
!python -B /drive/hypelcnn/classify/train_for_classification.py \
      --path=/drive/data \
      --loader_name=GULFPORTALTDataLoader \
      --importer=GeneratorImporter \
      --step=90000 \
      --max_evals=1 \
      --split_count=1 \
      --test_ratio=0 \
      --neighborhood=3 \
      --model_name=CONCNNModel \
      --algorithm_param_path=/drive/hypelcnn/nnmodel/modelconfigs/alg_param_concnn.json \
      --perform_validation \
      --save_checkpoint_steps=10000 \
      --base_log_path=/drive/class_log \
      --validation_steps=10000 \
      --train_ratio=125 \
      --augmentation_random_threshold=0.50 \
      --augment_data_with_shadow=simple

In [ ]:
!python -B /drive/hypelcnn/classify/infer_for_classification.py \
      --path=/drive/data \
      --loader_name=GULFPORTALTDataLoader \
      --importer=GeneratorImporter \
      --model_name=CONCNNModel \
      --neighborhood=3 \
      --batch_size=4000 \
      --base_log_path=/drive/class_log/episode_0/run_0/model.ckpt-0 \
      --output_path=/drive/class_log \
      --algorithm_param_path=/drive/hypelcnn/nnmodel/modelconfigs/alg_param_concnn.json

# Classic ML Tests

In [ ]:
!python -B /drive/hypelcnn/classify/classic_ml_trainer.py \
      --path=/drive/data \
      --loader_name=GULFPORTDataLoader \
      --neighborhood=1 \
      --batch_size=20000 \
      --split_count=1

# Learn Shadow via GAN

In [ ]:
!python -B /drive/hypelcnn/gan/gan_train_for_shadow.py \
      --path=/drive/data \
      --base_log_path=/drive/gan_log \
      --use_target_map=False \
      --step=100000 \
      --loader_name=GRSS2103DataLoader \
      --batch_size=32 \
      --validation_steps=1000 \
      --use_identity_loss=False \
      --cycle_consistency_loss_weight=10 \
      --gan_type=cut_x2y \
      --regularization_support_rate=0.0

In [ ]:
!python -B /drive/hypelcnn/gan/gan_infer_for_shadow.py \
      --path=/drive/data \
      --base_log_path=/drive/gan_log/model.ckpt-0 \
      --loader_name=AVONDataLoader \
      --neighborhood=0 \
      --gan_type=cycle_gan

In [ ]:
!python -B /drive/hypelcnn/gan/gan_infer_image_for_shadow.py \
      --path=/drive/data \
      --base_log_path=/drive/gan_log/model.ckpt-0 \
      --output_path=/drive/gan_log \
      --loader_name=AVONDataLoader \
      --make_them_shadow=shadow \
      --convert_all=False \
      --gan_type=cycle_gan

In [ ]:
from google.colab import files
files.download('/drive/hypelcnn/gan/log/shadow_image_rgb_shadow_.tif')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SR CycleGAN Training

In [ ]:
!python -B /drive/hypelcnn/gan/cycle_gann_sr_train.py \
      --path=/drive/data \
      --batch_size=100 \
      --max_number_of_steps=5000

In [ ]:
!python -B /drive/hypelcnn/gan/sr_gann_inference.py \
      --path=/drive/data \
      --checkpoint_path=/drive/hypelcnn/gan/log/model.ckpt-0 \
      --output_path=/drive/hypelcnn/gan/log